In [ ]:
from PIL import Image, ImageOps
import os,sys
import glob
attractionList = ['sana', 'nayeon', 'momo']
attractionFolderList = ['sana1', 'nayeon1', 'momo1']
for attractionFolder in attractionFolderList:
    image_dir = "./data/"+attractionFolder+"/"
    target_resize_dir = "./data_resize/"
    target_rotate_dir = "./data_rotate/"
    if not os.path.isdir(target_resize_dir):
        os.makedirs(target_resize_dir)
    if not os.path.isdir(target_rotate_dir):
        os.makedirs(target_rotate_dir)
    files = glob.glob(image_dir+"/*.*")
    print(image_dir)
    print(target_resize_dir)
    print(target_rotate_dir)
    print(files[0])
    print(files[0].split("/"))
    print(files[0].split("/")[-1])
    print(file.split("/")[-1].split(".")[0])
    print(len(files))
    count = 1;
    size = (256, 256)
    for file in files:
        im = Image.open(file)
        im = im.convert('RGB')
        print("i: ", count, im.format, im.size, im.mode, file.split("/")[-1])
        count+=1
        im = ImageOps.fit(im, size, Image.ANTIALIAS, 0, (0.5, 0.5))
        path = target_resize_dir+file.split("/")[-1].split(".")[0]
        print("path: " + path)
        im.save(path+".jpg", quality=100)
        
        

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Conv2D(30, (3, 3), strides=(1, 1), padding='same',
                 activation='relu', input_shape=(180,180,3), 
                 name='conv1'))
model.add(layers.MaxPooling2D((2, 2), strides=(3, 3), padding='same', 
                 name='pool1'))
model.add(layers.Conv2D(60, (3, 3), activation='relu', 
                 name='conv2'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same', 
                 name='pool2'))
model.add(layers.Conv2D(90, (3, 3), activation='relu', 
                 name='conv3'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same', 
                 name='pool3'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(2, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(lr=0.1),
              metrics=['accuracy'])


from keras.preprocessing.image import ImageDataGenerator
 
# train_datagen = ImageDataGenerator(rescale = 1./255)
 
# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.7,
                                   zoom_range=[0.9, 2.2],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)
 
training_set = train_datagen.flow_from_directory('./data',
                                                 shuffle=True,
                                                 seed=13,
                                                 target_size = (180, 180),
                                                 batch_size = 2,
                                                 class_mode = 'categorical',
                                                 subset="training")
validation_set = train_datagen.flow_from_directory('./data',
                                                 shuffle=True,
                                                 seed=13,
                                                 target_size = (180, 180),
                                                 batch_size = 2,
                                                 class_mode = 'categorical',
                                                 subset="validation")
#early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

csv_logger = CSVLogger('./log.csv', append=True, separator=';')

hist = model.fit_generator(training_set,
                         steps_per_epoch = 20,
                         epochs = 200,
                         validation_data = validation_set,
                         validation_steps = 10,
                         callbacks=[csv_logger])
from keras.models import load_model
 
model.save('cnn_attraction_keras_model.h5')
 
# output = classifier.predict_generator(test_set, steps=5)
# print(test_set.class_indices)
# print(output)
 
# 모델 평가하기
print("-- Evaluate --")
 
scores = model.evaluate_generator(
            validation_set,
            steps = 10)
 
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
 
# 모델 예측하기
print("-- Predict --")
 
output = model.predict_generator(
            validation_set,
            steps = 10)
print(validation_set.class_indices)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
 
print(output)
print(validation_set.filenames)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.callbacks import CSVLogger
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, utils

model = Sequential()

model.add(Conv2D(16, (48, 48), padding='same', use_bias=False, input_shape=(128, 256, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(32, (24, 24), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(64, (12, 12), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Conv2D(128, (6, 6), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.summary()

#sgd = optimizers.SGD(Ir=0.01, decay=1e-6, momentum=0.9, nestrov=True)
# Compiling the CNN
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
 
 
# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator
 
# train_datagen = ImageDataGenerator(rescale = 1./255)
 
# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.7,
                                   zoom_range=[0.9, 2.2],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)
 
training_set = train_datagen.flow_from_directory('./data',
                                                 shuffle=True,
                                                 seed=13,
                                                 target_size = (256, 256),
                                                 batch_size = 2,
                                                 class_mode = 'categorical',
                                                 subset="training")
validation_set = train_datagen.flow_from_directory('./data',
                                                 shuffle=True,
                                                 seed=13,
                                                 target_size = (256, 256),
                                                 batch_size = 2,
                                                 class_mode = 'categorical',
                                                 subset="validation")
#early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

csv_logger = CSVLogger('./log.csv', append=True, separator=';')

hist = model.fit_generator(training_set,
                         steps_per_epoch = 20,
                         epochs = 200,
                         validation_data = validation_set,
                         validation_steps = 10,
                         callbacks=[csv_logger])
from keras.models import load_model
 
model.save('cnn_attraction_keras_model.h5')
 
# output = classifier.predict_generator(test_set, steps=5)
# print(test_set.class_indices)
# print(output)
 
# 모델 평가하기
print("-- Evaluate --")
 
scores = model.evaluate_generator(
            validation_set,
            steps = 10)
 
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
 
# 모델 예측하기
print("-- Predict --")
 
output = model.predict_generator(
            validation_set,
            steps = 10)
print(validation_set.class_indices)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
 
print(output)
print(validation_set.filenames)



In [ ]:
from keras.models import load_model
model = load_model('./cnn_attraction_keras_model.h5')

# 모델 평가하기
print("-- Evaluate --")
 
scores = model.evaluate_generator(
            validation_set,
            steps = 10)
 
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
 
# 모델 예측하기
print("-- Predict --")
 
output = model.predict_generator(
            validation_set,
            steps = 10)
# print(output)
# print(validation_set.class_indices)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
 
# print(validation_set.filenames)
 
 
# 5. 학습과정 살펴보기
import matplotlib.pyplot as plt
 
fig, loss_ax = plt.subplots()
 
acc_ax = loss_ax.twinx()
 
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
#loss_ax.set_ylim([0.0, 0.5])
 
acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
#acc_ax.set_ylim([0.8, 1.0])
 
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')
 
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
 
plt.show()